# Unit 5 - Financial Planning

In [ ]:
# Imported the necessary libraries for project

import os

import requests

import pandas as pd

from dotenv import load_dotenv

import alpaca_trade_api as tradeapi

from MCForecastTools import MCSimulation

%matplotlib inline

In [ ]:
# Loaded .env enviroment variables

load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Created two variables called my_btc and my_eth. Set them equal to 1.2 and 5.3, respectively.

my_btc = 1.2

my_eth = 5.3

In [ ]:
# Prepared Crypto API URLs

btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"

eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [ ]:
# Fetched current BTC price in Canadian dollars

btc_price = requests.get(btc_url).json()

btc_price = btc_price["data"]["1"]["quotes"]["CAD"]["price"]

# Fetched current ETH price in Canadian dollars

eth_price = requests.get(eth_url).json()

eth_price = eth_price["data"]["1027"]["quotes"]["CAD"]["price"]

# Printed current value of BTC-CAD and ETH-CAD 

print(f"BTC-CAD: ${btc_price:0.2f}")

print(f"ETH-CAD: ${eth_price:0.2f}")

In [ ]:
# Computed the current portfolio value of cryptocurrencies.

btc_value = btc_price * my_btc

eth_value = eth_price * my_eth

# Printed current crypto wallet balance.

print(f"The current value of your {my_btc} BTC-CAD is ${btc_value:0.2f}")

print(f"The current value of your {my_eth} ETH-CAD is ${eth_value:0.2f}")




### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Created two variables named my_agg and my_spy and set them equal to 200 and 50, respectively.

my_agg = 200

my_spy = 50

In [ ]:
# Initialized Alpaca API key and secret.

alpaca_api_key = os.getenv("alpaca_akey")

alpaca_secret_key = os.getenv("alpaca_skey")

# Created the Alpaca API object.

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [ ]:
# Formatted current date as ISO format.

## 1000 trading days from 2021-06-13

start_date = pd.Timestamp("2017-06-22", tz="America/New_York").isoformat()

end_date = pd.Timestamp("2021-06-13", tz="America/New_York").isoformat()

# Created tickers variable.

tickers = ["AGG", "SPY"]

# Formatted timeframe to '1D' for Alpaca API.

timeframe = "1D"

# Retrieved current closing prices for SPY and AGG. Created a parameter to call the most recent 1000 days of data.

tickers_df = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000
).df


# Initialized tickers_df

tickers_df.sort_index(ascending=True)


In [ ]:
# Assessed data quality by determining the number of nulls from tickers_df

tickers_df.isnull().sum()

# Dropped records with null from tickers_df

tickers_df.dropna(inplace = True)

# Initialized last row from tickers_df to get closing prices of AGG and SPY

tickers_df.tail(1)

In [ ]:
# Intialized AGG and SPY closing prices

agg_closing_price = tickers_df.tail(1)["AGG"]["close"][0]

spy_closing_price = tickers_df.tail(1)["SPY"]["close"][0]

# Printed current AGG and SPY closing prices

print(f"Current AGG closing price: ${agg_closing_price}")

print(f"Current SPY closing price: ${spy_closing_price}")

In [ ]:
# Computed the current portfolio value of shares.

agg_value = agg_closing_price * my_agg

spy_value = spy_closing_price * my_spy

# Printed current value of shares

print(f"The current value of your {my_agg} AGG shares is ${spy_value:0.2f}")

print(f"The current value of your {my_spy} SPY shares is ${agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Created a variable of monthly household income with a value of "12000"

monthly_income = 12000

# Created a DataFrame called savings_df with two rows. Stored the total value in dollars of the crypto assets in the first row and the total value of the shares in the second row.

savings_df = pd.DataFrame([
    btc_value + eth_value,
    agg_value + spy_value
],
    columns=["Amount"],
    index=["Crypto", "Shares"]
)

# Initialized savings_df

savings_df

In [ ]:
# Used the savings_df DataFrame to plot a pie chart to visualize the composition of personal savings.

savings_df.plot.pie( y ="Amount", title = "Composition of Personal Savings", startangle=90)



In [ ]:
# Prepared variables for conditional statements to validate if the current savings are enough for an emergency fund. An ideal emergency fund should be equal to three times your monthly income.

# Created a variable for emergency fund.

emergency_fund = monthly_income * 3

# Calculated total amount of savings

total_savings = savings_df["Amount"].sum()

print(f"Total Savings: ${total_savings:0.2f}")

# Validated if total savings are enough to cover for emergency fund.

if total_savings > emergency_fund:

    print(f"Congratulations! You have more than enough savings to cover your emergency fund.")

elif total_savings == emergency_fund:

    print(f"Congratulations! You have reached your financial goal.")

else:

    print(f"Sorry! You do not have enough savings to cover your emergency fund. You need exactly ${emergency_fund - total_savings} to reach your financial goal.")


## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Collated and prepared variables to run Monte Carlo Simulations for AGG and SPY stock using recent historical data from "2016-06-14 to 2021-06-14"

## Take Note: Sample results may vary from the solution based on the time frame chosen.

# Retrieved current closing prices for SPY and AGG. 

# Dataframe was created on 2021-06-14 calculated 1000 trading days going backwards. Start date was set to "2017-07-23"

# Initialized tickers_df

tickers_df = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000
).df


# Initialized tickers_df

tickers_df.sort_index(ascending=True)


In [ ]:
# Created end date variable to retrieve the remaining historical data to complete the five year frame.

## tickers_df data includes historical data from (2017-06-23 to 2021-06-14) which is equivalent to exactly 1000 trading days

# Created a end_date2 variable 

end_date2 = pd.Timestamp("2017-06-21", tz='America/New_York').isoformat()

# Retrieved the remaining years' of historical data for AGG and SPY. Created a new variable tickers2_df and changed parameter to limit=260 to complete 1260 trading days back from "2021-06-14"

tickers2_df = api.get_barset(
    tickers,
    timeframe,
    end=end_date2,
    limit=260
).df

# Initialized missing historical data to complete five year frame.

tickers2_df.sort_index(ascending=True)

In [ ]:
# Checked tickers2_df for nulls.

tickers2_df.isnull().sum()

In [ ]:
# Combined tickers_df and tickers2_df to complete 5 year time frame from "2021-06-14". Created a new dataset labeled as combined_data_df.

combined_data_df = pd.concat([tickers_df, tickers2_df], axis = 0, join ="inner")

# Initialized combined_data_df.

combined_data_df.sort_index(ascending=True)


In [ ]:
# Checked data for nulls and duplicates. Prepared data for initialization.

combined_data_df.isnull().sum()

combined_data_df.dropna()

combined_data_df.duplicated().sum()

combined_data_df.duplicated(subset=None, keep='first')

# Initialized combined data frame of historical closing prices of AGG and SPY from "2016-06-06 to 2021-06-14"

combined_data_df.sort_index(ascending=True)

In [ ]:
# Configured a Monte Carlo simulation to forecast 30 years cumulative returns using combined_data_df. 

mc_30year = mcsimulation(
    portfolio_data = combined_data_df,
    weights = [.40,.60],
    num_simulation = 500,
    num_trading_days = 252*30
)

In [ ]:
# Printed the simulation input data of MC_30year.

mc_30year.portfolio_data.sort_index()

In [ ]:
# Initialized a Monte Carlo simulation to forecast 30 years cumulative returns.

mc_30year.calc_cumulative_return()


In [ ]:
# Plotted simulation outcomes of MC_30year dataframe. 

line_plot1 = mc_30year.plot_simulation()

In [ ]:
# Plotted probability distribution and confidence intervals of MC_30year dataframe. 

dist_plot1 = mc_30year.plot_distribution()

### Retirement Analysis

In [ ]:
# Created summary statistics from the Monte Carlo simulation results for MC_30year dataframe and labeled variable as "tbl1"

tbl1 = mc_30year.summarize_cumulative_return()

# Printed summary statistics of "tbl1"

tbl1

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Created a variable initial_investment1 with a value of 20000. 

initial_investment1 = 20000

# Created variables for the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of the $20,000 investment.

ci_lower1 = round(tbl1[8]*20000,2)

ci_upper1 = round(tbl1[9]*20000,2)


# Initialized results

print(f"There is a 95% chance that an initial investment of ${initial_investment1} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower1} and ${ci_upper1}")


### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Created a variable initial_investment2 with a value of 20000 * 1.5. 

initial_investment2 = 20000 * 1.5

# Created variables for the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of the $30,000 investment.

ci_lower2 = round(tbl1[8]*30000,2)

ci_upper2 = round(tbl1[9]*30000,2)

# Initialized results

print(f"There is a 95% chance that an initial investment of ${initial_investment2} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower2} and ${ci_upper2}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configured a Monte Carlo simulation to forecast 5 years cumulative returns.


mc_5year = mcsimulation(
    portfolio_data = combined_data_df,
    weights = [.40,.60],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Initialized a Monte Carlo simulation to forecast 5 years cumulative returns.

mc_5year.calc_cumulative_return()


In [ ]:
# Plotted simulation outcomes for MC_5year dataframe. 

line_plot2 = mc_5year.plot_simulation()

In [ ]:
# Plotted probability distribution and confidence intervals for MC_5year dataframe.

dist_plot2 = mc_5year.plot_distribution()

In [ ]:
# Created summary statistics from the Monte Carlo simulation results for MC_5year dataframe and labeled variable as "tbl2"

tbl2 = mc_5year.summarize_cumulative_return()

# Printed summary statistics of "tbl2"

tbl2

In [ ]:
# Created a variable initial investment3 with the value of 20000 * 3

initial_investment3 = 20000 * 3

# Created variables for the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000

ci_lower3 = round(tbl2[8]*60000,2)

ci_upper3 = round(tbl2[9]*60000,2)

# Initialized results

print(f"There is a 95% chance that an initial investment of ${initial_investment3} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower3} and ${ci_upper3}")

### Ten Years Retirement Option

In [ ]:
# Configured a Monte Carlo simulation to forecast 10 years cumulative returns using combined_data_df

mc_10year = mcsimulation(
    portfolio_data = combined_data_df,
    weights = [.40,.60],
    num_simulation = 500,
    num_trading_days = 252*10
)

In [ ]:
# Initialized a Monte Carlo simulation to forecast 10 years cumulative returns.

mc_10year.calc_cumulative_return()

In [ ]:
# Plotted simulation outcomes for MC_10year dataframe.

line_plot3 = mc_10year.plot_simulation()

In [ ]:
# Plotted probability distribution and confidence intervals for MC_10year dataframe.

dist_plot3 = mc_10year.plot_distribution()

In [ ]:
# Created summary statistics from the Monte Carlo simulation results for MC_10year dataframe and labeled it as "tbl3"

tbl3 = mc_10year.summarize_cumulative_return()

# Printed summary statistics of "tbl3"

tbl3

In [ ]:
# Created a variable for initial investment4 with the value of 20000 * 3 

initial_investment4 = 20000 * 3

# Created variables for the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000

ci_lower4 = round(tbl3[8]*60000,2)

ci_upper4 = round(tbl3[9]*60000,2)

# Initialized results

print(f"There is a 95% chance that an initial investment of ${initial_investment4} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower4} and ${ci_upper4}")